<font color = #00efff size = 5 face="標楷體"> 匯入函式庫(模組) </font>

In [1]:
#爬蟲相關套件
import requests
from bs4 import BeautifulSoup
import time

#資料處理與儲存相關套件
import pandas as pd
from IPython.display import display
import numpy as np
import re, random, time, os

#資料庫連結操作 pymysql.connections 
import pymysql
from sqlalchemy import create_engine

#------------------------------------------------
#Not every package has a __version__ attribute.
import sys
print('python version = ', sys.version)
from importlib.metadata import version
print('requests version = ', version('requests')) #same as (requests.__version__)
print('pandas version = ', pd.__version__)
print('pymysql version = ', version('pymysql')) 

python version =  3.9.16 (main, Jan 11 2023, 16:16:36) [MSC v.1916 64 bit (AMD64)]
requests version =  2.29.0
pandas version =  1.5.3
pymysql version =  1.0.2


<font color = #ff6f00 size = 5 face="Times New Roman"> 定義 PttCrawler 類，爬取 PTT 指定討論板多頁 HTML </font>
* 大部分代碼解釋請參閱 **爬取 PTT 多頁資料與貼文內容**

* Instance Attribute:
>1. theme : 由參數賦值，討論板主題。
>2. total_page : 由參數賦值，爬取總頁數。
>3. start_page :  PTT 討論版爬取起始頁。
>4. latest_page :　PTT 討論版的最新頁數，由 get_latest_pttpage() 賦值。
>5. last_page : PTT 討論版爬取末頁，預設是該討論版的`最新頁數`作為爬取末頁。
>6. ptt_urls :　PTT 討論板多頁網址，為列表 (list) 型態，由 assemble_multipage_urls()賦值。
>7. article_links : 文章內文的連結網址，為列表 (list) 型態。
>8. posts_info : 貼文資訊，包括文章標題、作者、發文日期、推文數、文章超連結、貼文內容，為字典(dictionary)型態。

* Instance Mehod:
>1. get_latest_pttpage() : 取得 PPT 論壇中欲爬取之討論板的最新頁數。
>2. assemble_multipage_urls() : 用來組合 PTT 多頁網址。
>3. fetch_html(url_list = None) : 發送傳入參數之 HTTP 請求，參數須為存放網址的列表(list)，預設參數為 PTT 網址列表，回傳值為存放 BeautifulSoup 結構樹的字典: {index : BeautifulSoup物件} 。
>4. get_posts_html():搜索 HTML 結構樹，並萃取出各貼文區塊標籤。
>4. get_posts_info(): 搜索並萃取出目標標籤，取得 PTT 貼文資訊，包括:文章標題、作者、發文日期、推文數、文章超連結、貼文內容，以字典形式回傳。
>5. get_posts_content(): 發送文章超連結請求，取得爬取到的文章內容，印出內文文字內容。

In [7]:
class Crawler_ptt:
    # 設定表頭偽裝成瀏覽器。(敘述太長以反斜線 '\' 分行方便閱讀)
    ptt_headers = {'over18':'1',          #一直向 server 回答滿 18 歲了 !
                    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)\
                    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84\
                    Safari/537.36'}
    
    def __init__(self, theme, total_page, last_page = None):
        self.theme = theme
        self.total_page = total_page
        self.latest_page = self.get_latest_pttpage()
        if last_page is None:
            self.last_page = self.latest_page
        else:
            self.last_page = last_page
        self.start_page = (self.last_page - self.total_page) + 1
        self.ptt_urls = self.assemble_multipage_urls()    #回傳組合好的網址賦值
        self.article_links = []    #存放貼文連結網址
        self.posts_info = {}    #存放貼文資訊
        
        
    def get_latest_pttpage(self): 
        """ 取得 PPT 論壇中欲爬取之討論板的最新頁數 """
        url = f'https://www.ptt.cc/bbs/{self.theme}/index.html'
        res = requests.get(url, headers = Crawler_ptt.ptt_headers)
        soup = BeautifulSoup(res.text, 'lxml')
        buttons = soup.find_all("a", class_ = "btn wide")
        for btn in buttons:
            if btn.text == '‹ 上頁':
                pre_url = btn["href"]
                pre_page = int(pre_url.split("/")[-1].split(".")[0].replace("index", ""))
                
                return pre_page + 1
        
            
    def assemble_multipage_urls(self):
        """ 用來組合討論版多頁網址，回傳網址列表賦值給 self.ptt_urls 
        """
        ptt_urls = []
        for p in range(self.start_page, self.last_page + 1):
            entire_url = f'https://www.ptt.cc/bbs/{self.theme}/index{p}.html'
            ptt_urls.append(entire_url)
            
        return ptt_urls
    
    
    def fetch_html(self, url_list = None):  
        """ 發送傳入參數之 HTTP 請求，參數須為存放網址的列表(list)型態，預設為 PTT 網址列表，
        回傳存放 BeautifulSoup 結構樹的字典: {index : BeautifulSoup物件} 
        """
        if url_list is None:
            url_list = self.ptt_urls    #預設參數傳入 self.ptt_urls
        else: 
            url_list = url_list
            
        bptree_dic = {}   #存放每頁網頁的 HTML 資料，{0:'BeautifulSoup1', 1:BeautifulSoup2,...}
        for i in range(len(url_list)):    
            try:
                r = requests.get(url_list[i], headers = Crawler_ptt.ptt_headers)
                bptree_dic[i] = BeautifulSoup(r.text, 'lxml') 
            except Exception as err:
                print('程式出現異常:', err)
                print(r.raise_for_status())
                print(r.reason)            
            else:
                # 確認請求是否成功
                if r.status_code == requests.codes.ok:  
                    print(f'{i+1}.成功取得 {url_list[i]} 資料')
                else:  
                    print(f'{i+1}.{url_list[i]} 取得失敗')
            if i % 50 == 0:time.sleep(random.uniform(2,4))

        return bptree_dic
    
    
    def get_posts_html(self):
        """ 搜索 HTML 結構樹，並萃取出各貼文區塊之標籤。
        """
        post_divs = []    #存放每篇貼文 html 區塊
        ptt_bp_dic = self.fetch_html()   #調用 fetch_html() 實體方法
        for k in ptt_bp_dic:    #一一取出字典中每頁 BeautifulSoup 資料  
            # 找出各篇貼文之 html 區塊
            post_divs += ptt_bp_dic[k].select('div.r-ent')
            
        return post_divs
    
            
    def get_posts_info(self):
        """取得 PTT 貼文資訊，包括:文章標題、作者、發文日期、推文數、文章超連結、文章內容，
        以字典形式回傳。
        """
        posts_html = self.get_posts_html()
        num = 1
        for post in posts_html:    
            if post != None:    #確認 post 不為空值
                title = post.find('div', class_='title').text.strip()
                link = post.select_one('.title a') 
                author = post.find('div', 'author').text.strip()
                post_date = post.find('div', 'date').text.strip()
                push_num = post.find('div', 'nrec').text.strip()
                if link is not None:
                    self.article_links.append(f'https://www.ptt.cc{link.get("href")}')
                else:
                    self.article_links.append('No link found')
                self.posts_info[f'No.{num}'] = {  
                    "title": title,
                    "author": author,
                    "post_date": post_date,
                    "push_num": push_num,
                    "article_links": link,
                    "content": '\n'}
                num += 1
            else: print('posts_html 為 None')
        self.get_posts_content()
      
        return self.posts_info
    
    
    def get_posts_content(self):
        """ 取得文章內容，列印出每一篇貼文文章內容。 
        """
        links_bp_dic = self.fetch_html(self.article_links)   #調用實體方法 fetch_html()，參數為文章超連結串列
        if len(self.posts_info) == len(self.article_links):    
            for l in links_bp_dic:    
                content_div = links_bp_dic[l].find('div', id = 'main-content')
                content_list = list(content_div)  #將 html 結構樹轉為列表
                for item in content_list:
                    if isinstance(item, str):
                        self.posts_info[f'No.{l+1}']['content'] += f'{item.strip()}'
                    elif item.has_attr('href'):
                        self.posts_info[f'No.{l+1}']['content'] += f'{item.text}' 
                print(f"第 {l+1} 篇: {self.posts_info[f'No.{l+1}']['content']}")
                print('=='*50)

        else:print('貼文數量與貼文連結數量不一致')
            
        

In [8]:
if __name__ == "__main__":
    #創建實體物件 
    cptt = Crawler_ptt('Soft_Job', 2)     
    #調用 get_posts_info()方法更新 self.article_links, self.posts_info，以及取得字典形式的貼文資訊，
    cptt.get_posts_info()
    #以表格形式顯示貼文資訊
    posts_df = pd.DataFrame(cptt.posts_info).T
    print(posts_df.info())
    display(posts_df)
    

1.成功取得 https://www.ptt.cc/bbs/Soft_Job/index1754.html 資料
2.成功取得 https://www.ptt.cc/bbs/Soft_Job/index1755.html 資料
1.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1683519326.A.3F7.html 資料
2.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1683531484.A.FE0.html 資料
3.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1683535626.A.5F2.html 資料
4.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1683536591.A.983.html 資料
5.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1683549816.A.BBA.html 資料
6.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1683565711.A.B54.html 資料
7.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1683596444.A.343.html 資料
8.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1683606691.A.3AD.html 資料
9.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1683637949.A.4AA.html 資料
10.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1683640295.A.2BA.html 資料
11.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1683707371.A.784.html 資料
12.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1683716049.A.227.html 資料
13.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1683727551.A.278.html 資料
14.成功取得 https://www.ptt.

<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, No.1 to No.27
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          27 non-null     object
 1   author         27 non-null     object
 2   post_date      27 non-null     object
 3   push_num       27 non-null     object
 4   article_links  26 non-null     object
 5   content        27 non-null     object
dtypes: object(6)
memory usage: 1.5+ KB
None


,title,author,post_date,push_num,article_links,content
No.1,Re: [請益] 好像不常聽到工程師研究程式交易？,joywilliamjo,5/08,,[Re: [請益] 好像不常聽到工程師研究程式交易？],\n有做的其實比想像中的多，而且說賺的很大的也有\n不常聽到的原因大概是因為你就跟那些人不在...
No.2,Re: [請益] 好像不常聽到工程師研究程式交易？,y2468101216,5/08,2,[Re: [請益] 好像不常聽到工程師研究程式交易？],\n其實這跟股板比較有關。\n\n簡單說幾個我的觀點。\n\n1. 會賺錢的方法自己賺都來不...
No.3,Re: [請益] 好像不常聽到工程師研究程式交易？,qdullqidiot,5/08,2,[Re: [請益] 好像不常聽到工程師研究程式交易？],\n首先我就是讀商科 後來對寫程式有興趣\n所以跳槽工程師\n剛好又接觸到程式交易有興趣\n...
No.4,Re: [請益] 接家業or回台當軟工,apage,5/08,16,[Re: [請益] 接家業or回台當軟工],\n我今年43歲，36歲從大陸回台灣，跟香港的姑姑一起合作開公司\n40歲決裂自己工作，現在...
No.5,Re: [請益] 好像不常聽到工程師研究程式交易？,capita,5/08,5,[Re: [請益] 好像不常聽到工程師研究程式交易？],\n程式交易不少人在做，但工程師做程式交易的確實很少。\n\n簡單來說，程式交易的金融部份，...
No.6,Re: [請益] 好像不常聽到工程師研究程式交易？,sorryla,5/09,1,[Re: [請益] 好像不常聽到工程師研究程式交易？],\n問題在於你想要用程式幫到你交易的哪個部分？\n\n如果你是要快進快出賺極短線的，你打得贏...
No.7,[徵才] 全遠端美商 Swifteam,coedschool,5/09,15,[[徵才] 全遠端美商 Swifteam],\n公司名稱，統編(中華民國以外註冊可免填):\n\nSwifteam\n\nSwiftea...
No.8,[心得] 軟體考古系列：Redis,eliang,5/09,97,[[心得] 軟體考古系列：Redis],\n我一直覺得 Redis 在資料庫世界裡獨具一格。其他多數資料庫的中心思想不是表格就\n是...
No.9,[討論] 閒聊文-為什麼前端這麼卷呀？,secretfly,5/09,31,[[討論] 閒聊文-為什麼前端這麼卷呀？],\n對岸的詞語 卷 好像就是一個很競爭的意思\n\n我發現好像台灣這邊的前端也差不多\n\n...
No.10,[請益] 應該轉正還是去上課呢？（非本科）,denny41606,5/09,30,[[請益] 應該轉正還是去上課呢？（非本科）],\n各位好，之前曾經問過offer選擇的事情，很感謝版友們給我的回饋\n目前在猶豫是否應該在...


<font color= #0fe9de size= 5 face="標楷體"> 將 DataFrame 匯入資料庫 </font>

